In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/shivamb/netflix-shows/netflix_titles.csv


# Business Understanding

Explore Netflix content growth, trends, ratings, and clusters.

Build recommendation system for similar content.

Predict type (Movie/TV Show).

# Data Loading & Understanding

In [53]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv('/kaggle/input/datasets/shivamb/netflix-shows/netflix_titles.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


8807 rows, 12 columns.

Missing values in director, cast, country, date_added, rating, duration.

In [54]:
df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [55]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


# Data Cleaning & Feature Engineering

#### Clean date_added

In [56]:
df['date_added'] = pd.to_datetime(df['date_added'].str.strip(), errors='coerce')
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

We need datetime for trends over years.
 year_added and month_added extracted.

#### fill missing values 

In [57]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
year_added        10
month_added       10
dtype: int64

In [58]:
cols_to_fill = ['director','cast','country','rating','duration']
df[cols_to_fill] = df[cols_to_fill].fillna('Unknown')

In [59]:
df.isna().sum()

show_id          0
type             0
title            0
director         0
cast             0
country          0
date_added      10
release_year     0
rating           0
duration         0
listed_in        0
description      0
year_added      10
month_added     10
dtype: int64

#### Extract numeric duration

In [60]:
df['duration_int'] = pd.to_numeric(df['duration'].str.extract(r'(\d+)')[0], errors='coerce')

We need numeric duration for ML and clustering.
so Duration column converted to integers (minutes or seasons).

#### feature engineering

In [61]:
df['content_age_when_added'] = df['year_added'] - df['release_year']

Helps understand whether Netflix adds old/new content.
so: New column with age of content at the time of adding.

#### Encode type column

In [62]:
df['is_movie'] = df['type'].apply(lambda x: 1 if x=='Movie' else 0)

For ML classification.
so: Binary column: 1=Movie, 0=TV Show.

# Interactive Content Growth Over Years

In [69]:
fig = px.histogram(df, x='year_added', color='type',
                   title='Content Growth Over Years',
                   labels={'year_added':'Year Added','count':'Number of Titles','type':'Content Type'},
                   barmode='group')
fig.update_layout(xaxis=dict(dtick=1))
fig.show()

To visualize how Netflix content grew over time.
so: Clear growth surge after 2016, TV Shows increasing faster than Movies.

#### Average Content Age When Added

In [70]:
age_df = df.groupby('year_added')['content_age_when_added'].mean().reset_index()
fig = px.line(age_df, x='year_added', y='content_age_when_added', markers=True,
              title='Average Content Age When Added',
              labels={'year_added':'Year Added', 'content_age_when_added':'Age'})
fig.show()

See if Netflix focuses on new releases or older content.
so: Shows Netflix mostly adds recent content, but some older classics exist.

#### Ratings Trend

In [71]:
rating_df = df[df['rating']!='Unknown']
fig = px.histogram(rating_df, x='year_added', color='rating',
                   barmode='group',
                   title='TV Ratings Trend',
                   labels={'year_added':'Year Added','count':'Number of Titles','rating':'Rating'})
fig.update_layout(xaxis=dict(dtick=1))
fig.show()

Understand distribution of ratings over years.
so: TV-MA and TV-14 increasing; PG and TV-Y mostly stable.

# Clustering Content

In [66]:
features = df[['release_year','duration_int','content_age_when_added']].fillna(0)
scaler = StandardScaler()
scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(scaled)

fig = px.histogram(df, x='cluster', color='type', barmode='group',
                   title='Content Clusters Distribution',
                   labels={'cluster':'Cluster','count':'Number of Titles','type':'Content Type'})
fig.show()

To segment content into groups with similar features.
so: 4 clusters reveal patterns in duration, release year, and content type.

# Machine Learning: Predict Content Type

In [67]:
features = df[['release_year','duration_int','content_age_when_added']].fillna(0)
target = df['is_movie']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
print("Logistic Regression Results:")
print(classification_report(y_test, lr.predict(X_test)))

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print("Random Forest Results:")
print(classification_report(y_test, rf.predict(X_test)))

Logistic Regression Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       548
           1       1.00      1.00      1.00      1214

    accuracy                           1.00      1762
   macro avg       1.00      1.00      1.00      1762
weighted avg       1.00      1.00      1.00      1762

Random Forest Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       548
           1       1.00      1.00      1.00      1214

    accuracy                           1.00      1762
   macro avg       1.00      1.00      1.00      1762
weighted avg       1.00      1.00      1.00      1762



Predict whether a title is a Movie or TV Show based on features.
so: Random Forest performs better; features like duration and release_year most important.

# Recommendation System (TF-IDF + Cosine Similarity)

In [72]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])
cosine_sim = cosine_similarity(tfidf_matrix)
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def recommend(title):
    idx = indices.get(title)
    if idx is None:
        return ["Title not found!"]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

# Example interactive recommendation
import ipywidgets as widgets
title_widget = widgets.Combobox(
    placeholder='Type title here',
    options=list(df['title'].unique()),
    description='Title:',
    ensure_option=True,
    continuous_update=False
)

def show_recommendations(title):
    recs = recommend(title)
    print("Recommended Titles:")
    for t in recs:
        print("-", t)

widgets.interact(show_recommendations, title=title_widget)

interactive(children=(Combobox(value='', continuous_update=False, description='Title:', ensure_option=True, op…

<function __main__.show_recommendations(title)>

To suggest similar content based on description.
so: Works interactively; typing any title gives 5 recommended titles.

# Summary Insights:

Massive content growth after 2016.

TV Shows growing faster than Movies.

Netflix mostly adds recent content, but older classics exist.

Ratings trend shows increase in mature content.

Clustering reveals natural content groups.

Random Forest predicts type accurately.

TF-IDF recommendation works interactively.